In [22]:
import numpy as np
from sympy import *
from sympy.polys.monomials import monomial_divides, term_div

In [89]:
#Signature is stored as tuple (m,i) where m is the monomial and i is the index
#Poly is stored as a polynomial ie. x**a + x**b +...
class LabeledPolynomial:
    def __init__(self, sign, poly,num, order = 'lex'):
        self.poly = poly
        self.sign = sign
        self.order = order
        self.num = num
    
def sig_cmp(f,g,order = 'lex'):
    if (f[1] > g[1]):
        return -1
    if (f[1] == g[1]):
        temp = f[0]+g[0]
        if not LM(temp,order = order) == f[0]:
            return -1
    return 1

def lbp_sub(F,G):
    if(sig_cmp(F.sign,G.sign,order = F.order)) < 0:
        sign = G.sign
        num = G.num
    else:
        sign = F.sign
        num = F.num
    poly = F.poly-G.poly
    return(LabeledPolynomial(sign,poly,num,F.order))
    
def lbp_mult(F,m):
    sign = (F.sign[0]*m,F.sign[1])
    poly = expand(F.poly*m)
    num = F.num
    return(LabeledPolynomial(sign,poly,num,F.order))


def CP(F,G):
    LM1 = LM(F.poly,order = F.order)
    LT1 = LT(F.poly,order = F.order)
    LM2 = LM(G.poly,order = G.order)
    LT2 = LT(G.poly,order = G.order)
    numerator = lcm(LM1,LM2)
    u = simplify(numerator/LT1)
    v = simplify(numerator/LT2)
    return((u,F,v,G))
    
def spoly(pair):
    lbp1 = lbp_mult(pair[1],pair[0])
    lbp2 = lbp_mult(pair[3],pair[2])
    return(lbp_sub(lbp1,lbp2))

def divisible_or_rewritable(F,B):
    for G in B:
        if G.sign[1] > F.sign[1]:
            lpp = LM(G.poly,order = G.order)
            if rem(lpp,F.sign[0]):
                return True
        if G.sign[1] == F.sign[1]:
            if F.num < G.num:
                if rem(G.sign[0],F.sign[0]):
                    return True
    return False

def f5_reduce(F,B):
    if not F.poly == 0:
        LM1 = LM(F.poly,order = F.order)
        is_reducible = False
        for G in B:
            LM2 = LM(G.poly,order = G.order)
            if rem(LM2,LM1) == 0:
                LT1 = LT(F.poly,order = F.order)
                LT2 = LT(G.poly,order = G.order)
                v = simplify(LT1/LT2)
                vG = lbp_mult(G,v)
                if sig_cmp(vG.sign,F.sign) < 0:
                    if not divisible_or_rewritable(vG,B):
                        is_reducible = True
                        break
        if is_reducible:
            return(f5_reduce(lbp_sub(F,vG),B))
        else:
            return(F)
    else:
        return(LabeledPolynomial(1,0,0))

def f5_grobner(P,order = 'lex'):
    B = []
    for i,p in enumerate(P):
        F = LabeledPolynomial((1,i),p,i,order = order)
        B.append(F)
    n = len(B)
    pairs = []
    for i in range(n-1):
        for j in range(i+1,n):
            pairs.append(CP(B[i],B[j]))
    while len(pairs) > 0:
        pair = pairs.pop()
        uF = lbp_mult(pair[1],pair[0])
        if not divisible_or_rewritable(uF,B):
            vG = lbp_mult(pair[3],pair[2])
            if not divisible_or_rewritable(vG,B):
                S = spoly(pair)
                r = f5_reduce(S,B)
                if not r.poly == 0:
                    for b in B:
                        pairs.append(CP(r,b))
                    B.append(r)
    return([b.poly for b in B])
            

In [195]:
def spoly_orig(f1,f2,order = 'lex'):
    LM1 = LM(f1,order = order)
    LT1 = LT(f1,order = order)
    LM2 = LM(f2,order = order)
    LT2 = LT(f2,order = order)
    numerator = lcm(LM1,LM2)
    S = numerator/LT1*f1 - numerator/LT2 * f2
    return(expand(S))

def minimize_GBasis(G, order = 'lex'):
    for i,g in enumerate(G):
        G[i] = g/LC(g,order = order)
    n = len(G)
    G2 = []
    for g in G:
        H = [f for f in G]
        H.remove(g)
        H = [LT(f,order = order) for f in H]
        LTg = LT(g,order = order)
        q,r = reduced(LTg, H, order = order)
        if not r == 0:
            G2.append(g)
    return(G2)

def reduce_GBasis(G, order = 'lex'):
    G = minimize_GBasis(G, order = order)
    for i,g in enumerate(G):
        H = [f for f in G if not f == g]
        q,r = reduced(g,H, order = order)
        G[i] = r
    return(G)

def red_groebner(G,order = 'lex'):
    F = G
    H = []
    while F:
        f0 = F.pop()
        r = [monomial_divides(f.LM(),f0.LM()) for f in F+H]
        add = True
        for t in r:
            if t:
                add = False
                break
        if add:
            H.append(f0)
    return(reduction(H,order = order))

def reduction(P,order = 'lex'):
    Q = []
    for i,p in enumerate(P):
        q,h = reduced(p,P[:i]+P[i+1:],order = order)
        if h:
            Q.append(h)
    return [p.monic() for p in Q]

def GrobAdd_Buchberger(G,f, order = 'lex'):
    Q = [f]
    while len(Q) > 0:
        g = Q.pop(0)
        for g2 in G: 
            S = spoly(g, g2, order = order)
            q,r = reduced(S,G,order = order)
            if not r == 0: 
                Q.append(r)
        G.append(f)
    G = reduce_GBasis(G,order = order)
    return(G)

In [202]:
x,y = var('x,y')
F = [x*y-x, x**2-y]
G = groebner(F,order = 'lex',domain = 'QQ')
print(list(G))

[x**2 - y, x*y - x, y**2 - y]


In [203]:
B = f5_grobner(F,order = 'lex')
print(B)

[x*y - x, x**2 - y, y**2 - y]


In [204]:
x,y = var('x,y')
F = [2*x**2-4*x+y**2-4*y+3, x**2-2*x+3*y**2-12*y+9]
G = groebner(F,order = 'lex',domain = 'QQ')
print(list(G))

[x**2 - 2*x, y**2 - 4*y + 3]


In [199]:
B = f5_grobner(F,order = 'lex')
print(B)

[2*x**2 - 4*x + y**2 - 4*y + 3, x**2 - 2*x + 3*y**2 - 12*y + 9, -5*y**2/2 + 10*y - 15/2, -4*x**2*y + 3*x**2 + 2*x*y**2 - 3*y**4 + 12*y**3 - 9*y**2, -x*y**2/2 + 2*x*y - 3*x/2 + 3*y**4/4 - 6*y**3 + 117*y**2/8 - 21*y/2 + 9/8, -3*y**4/2 + 12*y**3 - 117*y**2/4 + 21*y - 9/4, -3*x*y**4/2 + 12*x*y**3 - 117*x*y**2/4 + 21*x*y - 9*x/4]


In [200]:
B = [Poly(b) for b in B]
red_groebner(B,order = 'lex')

[Poly(x*y**2 - 4*x*y + 3*x - 3/2*y**4 + 12*y**3 - 117/4*y**2 + 21*y - 9/4, x, y, domain='QQ'),
 Poly(x**2 - 2*x + 1/2*y**2 - 2*y + 3/2, x, y, domain='QQ')]

In [201]:
print(B)

[]
